# RNN

In [ ]:
# !pip install tensorflow==2.14 openpyxl==3.1.2 pandas numpy

!pip show tensorflow # 2.14.0
!pip show openpyxl   # 3.1.2

In [2]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
text = """곱다
경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다.\n
가는 말이 고와야 오는 말이 \n"""

In [4]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
vocab_size = len(tokenizer.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 12


In [5]:
print(tokenizer.word_index)

{'말이': 1, '곱다': 2, '경마장에': 3, '있는': 4, '뛰고': 5, '있다': 6, '그의': 7, '법이다': 8, '가는': 9, '고와야': 10, '오는': 11}


In [6]:
sequences = list()
for line in text.split('\n'): # 줄 바꿈 문자를 기준으로 문장 토큰화
    encoded = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[: i+1]
        sequences.append(sequence)

print('학습에 사용할 샘플의 개수: %d' % len(sequences))

학습에 사용할 샘플의 개수: 10


In [7]:
print(sequences)

[[3, 4], [3, 4, 1], [3, 4, 1, 5], [3, 4, 1, 5, 6], [7, 1], [7, 1, 8], [9, 1], [9, 1, 10], [9, 1, 10, 11], [9, 1, 10, 11, 1]]


In [8]:
max_len = max(len(l) for l in sequences) # 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 5


In [9]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

In [10]:
print(sequences)

[[ 0  0  0  3  4]
 [ 0  0  3  4  1]
 [ 0  3  4  1  5]
 [ 3  4  1  5  6]
 [ 0  0  0  7  1]
 [ 0  0  7  1  8]
 [ 0  0  0  9  1]
 [ 0  0  9  1 10]
 [ 0  9  1 10 11]
 [ 9  1 10 11  1]]


In [11]:
sequences = np.array(sequences)
X = sequences[:, :-1]
y = sequences[:, -1]

In [12]:
sequences.shape

(10, 5)

In [13]:
print(X)

[[ 0  0  0  3]
 [ 0  0  3  4]
 [ 0  3  4  1]
 [ 3  4  1  5]
 [ 0  0  0  7]
 [ 0  0  7  1]
 [ 0  0  0  9]
 [ 0  0  9  1]
 [ 0  9  1 10]
 [ 9  1 10 11]]


In [14]:
print(y)

[ 4  1  5  6  1  8  1 10 11  1]


In [15]:
y = to_categorical(y, num_classes=vocab_size)

In [16]:
print(y)

[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


## 2) 모델 설계하기
RNN 모델에 데이터를 훈련시키기

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

In [18]:
embedding_dim = 10
hidden_units = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(SimpleRNN(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
1/1 - 2s - loss: 2.4921 - accuracy: 0.0000e+00 - 2s/epoch - 2s/step
Epoch 2/200
1/1 - 0s - loss: 2.4802 - accuracy: 0.0000e+00 - 10ms/epoch - 10ms/step
Epoch 3/200
1/1 - 0s - loss: 2.4683 - accuracy: 0.1000 - 10ms/epoch - 10ms/step
Epoch 4/200
1/1 - 0s - loss: 2.4564 - accuracy: 0.1000 - 12ms/epoch - 12ms/step
Epoch 5/200
1/1 - 0s - loss: 2.4442 - accuracy: 0.3000 - 11ms/epoch - 11ms/step
Epoch 6/200
1/1 - 0s - loss: 2.4317 - accuracy: 0.3000 - 10ms/epoch - 10ms/step
Epoch 7/200
1/1 - 0s - loss: 2.4188 - accuracy: 0.3000 - 14ms/epoch - 14ms/step
Epoch 8/200
1/1 - 0s - loss: 2.4054 - accuracy: 0.3000 - 13ms/epoch - 13ms/step
Epoch 9/200
1/1 - 0s - loss: 2.3914 - accuracy: 0.4000 - 13ms/epoch - 13ms/step
Epoch 10/200
1/1 - 0s - loss: 2.3768 - accuracy: 0.4000 - 13ms/epoch - 13ms/step
Epoch 11/200
1/1 - 0s - loss: 2.3616 - accuracy: 0.4000 - 13ms/epoch - 13ms/step
Epoch 12/200
1/1 - 0s - loss: 2.3455 - accuracy: 0.4000 - 11ms/epoch - 11ms/step
Epoch 13/200
1/1 - 0s - loss: 2.3

In [19]:
def sentence_generation(model, tokenizer, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word
    sentence = ''

    # n번 반복
    for _ in range(n):
        # 현재 단어에 대한 정수 인코딩과 패딩
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=5, padding='pre')
        # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for word, index in tokenizer.word_index.items(): 
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면 break
            if index == result:
                break

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' '  + word

        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence

In [20]:
print(sentence_generation(model, tokenizer, '경마장에', 4))

경마장에 있는 말이 뛰고 있다


In [21]:
print(sentence_generation(model, tokenizer, '가는', 5))

가는 말이 고와야 오는 말이 고와야


# LSTM을 이용하여 텍스트 생성하기

In [22]:
import numpy as np
import pandas as pd
import pandas as pad_sequences
from string import punctuation

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [23]:
df = pd.read_excel(r'C:/Users/GJAISCHOOL/Desktop/X_filter/Algorithm/dataset/good_sentence.xlsx')
df.head()

,Sentence,label
0,좌배 까는건 ㅇㅂ,1
1,집에 롱 패딩만 세 개다. 10년 더 입어야지 ㅋㅋ,0
2,개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아,1
3,세탁이라고 봐도 된다,0
4,애새끼가 초딩도 아니고 ㅋㅋㅋㅋ,1


In [24]:
print('열의 개수 : ', len(df.columns))
print(df.columns)

열의 개수 :  2
Index(['Sentence', 'label'], dtype='object')


In [25]:
print(df['Sentence'].isnull().values.any())

False


In [26]:
headline = []
# 헤드라인의 값들을 리스트로 저장
headline.extend(list(df.Sentence.values))
headline[:5]

['좌배 까는건 ㅇㅂ',
 '집에 롱 패딩만 세 개다. 10년 더 입어야지 ㅋㅋ',
 '개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아',
 '세탁이라고 봐도 된다',
 '애새끼가 초딩도 아니고 ㅋㅋㅋㅋ']

In [27]:
print('총 샘플의 개수 : {}'.format(len(headline)))

총 샘플의 개수 : 28737


In [28]:
headline = [word for word in headline if word != "Unkown"]
print('노이즈값 제거 후 샘플의 개수 : {}'.format(len(headline)))

노이즈값 제거 후 샘플의 개수 : 28737


In [29]:
headline[:5]

['좌배 까는건 ㅇㅂ',
 '집에 롱 패딩만 세 개다. 10년 더 입어야지 ㅋㅋ',
 '개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아',
 '세탁이라고 봐도 된다',
 '애새끼가 초딩도 아니고 ㅋㅋㅋㅋ']

In [30]:
def repreprocessing(raw_sentence):
    preproceseed_sentence = raw_sentence.encode("utf8").decode("ascii",'ignore')
    # 구두점 제거와 동시에 소문자화
    return ''.join(word for word in preproceseed_sentence if word not in punctuation).lower()

preprocessed_headline = [repreprocessing(x) for x in headline]
preprocessed_headline[:5]

['  ', '     10   ', '        ', '  ', '   ']

In [31]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_headline)
vocab_size = len(tokenizer.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 1059


In [32]:
sequences = list()

for sentence in preprocessed_headline:
    # 각 샘플에 대한 정수 인코딩
    encoded = tokenizer.texts_to_sequences([sentence])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:11]

[[6, 16],
 [10, 10],
 [10, 10, 10],
 [235, 236],
 [235, 236, 163],
 [1, 5],
 [79, 89],
 [79, 89, 32],
 [363, 13],
 [237, 364],
 [164, 15]]

In [33]:
index_to_word = {}
for key, value in tokenizer.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key

print('빈도수 상위 582번 단어 : {}'.format(index_to_word[582]))

빈도수 상위 582번 단어 : httpswwwilbecomview11226188404c11226214943comm11226214943


In [34]:
max_len = max(len(l) for l in sequences)
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 53


In [35]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  6 16]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0 10 10]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0 10 10 10]]


In [36]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

In [37]:
print(X[: 3])

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  6]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0 10]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0 10 10]]


In [38]:
print(y[:3])

[16 10 10]


In [39]:
y = to_categorical(y, num_classes=vocab_size)

In [40]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM


In [41]:
embedding_dim = 10
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=2000, verbose=2)


Epoch 1/2000
48/48 - 4s - loss: 6.1327 - accuracy: 0.0588 - 4s/epoch - 79ms/step
Epoch 2/2000
48/48 - 0s - loss: 5.3501 - accuracy: 0.0673 - 356ms/epoch - 7ms/step
Epoch 3/2000
48/48 - 0s - loss: 5.2583 - accuracy: 0.0719 - 357ms/epoch - 7ms/step
Epoch 4/2000
48/48 - 0s - loss: 5.2206 - accuracy: 0.0667 - 366ms/epoch - 8ms/step
Epoch 5/2000
48/48 - 0s - loss: 5.1683 - accuracy: 0.0719 - 389ms/epoch - 8ms/step
Epoch 6/2000
48/48 - 0s - loss: 5.1178 - accuracy: 0.0686 - 483ms/epoch - 10ms/step
Epoch 7/2000
48/48 - 1s - loss: 5.0666 - accuracy: 0.0706 - 565ms/epoch - 12ms/step
Epoch 8/2000
48/48 - 1s - loss: 5.0090 - accuracy: 0.0686 - 701ms/epoch - 15ms/step
Epoch 9/2000
48/48 - 0s - loss: 4.9540 - accuracy: 0.0699 - 431ms/epoch - 9ms/step
Epoch 10/2000
48/48 - 0s - loss: 4.9078 - accuracy: 0.0680 - 485ms/epoch - 10ms/step
Epoch 11/2000
48/48 - 1s - loss: 4.8668 - accuracy: 0.0706 - 575ms/epoch - 12ms/step
Epoch 12/2000
48/48 - 1s - loss: 4.8243 - accuracy: 0.0706 - 572ms/epoch - 12ms/st

In [42]:
def sentence_generation(model, tokenizer, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word
    sentence = ''

    # n번 반복
    for _ in range(n):
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=max_len-1, padding='pre')

        # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for word, index in tokenizer.word_index.items(): 
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
            if index == result:
                break

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' '  + word

        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence


In [43]:
print(sentence_generation(model, tokenizer, '병신엿같네', 10))


병신엿같네 mbc 40 215 40 215 40 215 40 215 40


In [44]:
print(sentence_generation(model, tokenizer, 'how', 10))

how mbc 40 215 40 215 40 215 40 215 40
